# import dependencies

In [1]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
!pip install pykitti
import pykitti
import cv2
from matplotlib.patches import Circle
import csv

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import sys
!pip install torchvision
import sys
!{sys.executable} -m pip install torchsummary
!{sys.executable} -m pip install pytorchvis
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
!pip install pytorchvis
!pip install torchsummary
from torchsummary import summary
from pytorchvis.visualize_layers import VisualizeLayers
from PIL import Image as PImage
import io
import math
from scipy.spatial import distance as dist


In [2]:
# Run on Google Colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Load Kitti dataset

In [0]:
# Path to KITTI directory
basedir = '/content/drive/2011_09_26_drive_0001_sync'
date = '2011_09_26'
drive = '0001'

dataset = pykitti.raw(basedir, date, drive)

## Camera displacement utils

In [0]:
def latToScale(lat):
# compute mercator scale from latitude
    return math.cos(lat * math.pi / 180)

def latlonToMercator(lat,lon,scale):
# converts lat/lon coordinates to mercator coordinates using mercator scale
    er = 6378137 # earth radius
    mx = scale * lon * math.pi * er / 180
    my = scale * er * math.log( math.tan((90+lat) * math.pi / 360) )
    return mx, my

def convertOxtsToPose(oxts):
    scale = latToScale(oxts[0][0])
    pose = []
    Tr_0_inv = np.zeros(shape=(4,4))
    Tr = np.zeros(shape=(4,4))
    t = np.zeros(shape=(3, 1))
    for i in range(len(oxts)):
        if not oxts[i]:
            pose.append([])
            continue

        # translation vector 
        t[0,0], t[1,0] = latlonToMercator(oxts[i][0],oxts[i][1],scale)
        t[2,0] = oxts[i][2]

        # rotation matrix (OXTS RT3000 user manual, page 71/92)
        rx = oxts[i][3] # roll
        ry = oxts[i][4] # pitch
        rz = oxts[i][5] # heading 
        # base => nav  (level oxts => rotated oxts)
        Rx = np.matrix([[1, 0, 0], [0, math.cos(rx), -math.sin(rx)], [0, math.sin(rx), math.cos(rx)]]) 
        # base => nav  (level oxts => rotated oxts)
        Ry =  np.matrix([[math.cos(ry), 0, math.sin(ry)], [0, 1, 0], [-math.sin(ry), 0, math.cos(ry)]]) 
        # base => nav  (level oxts => rotated oxts)
        Rz = np.matrix([[math.cos(rz), -math.sin(rz), 0], [math.sin(rz), math.cos(rz), 0], [0, 0, 1]]) 
        R  = Rz*Ry*Rx

        # print(R)
        # print(t)
        Tr[:3, :3] = R
        Tr[:3, 3:] = t
        Tr[3,3] = 1
        # normalize translation and rotation (start at 0/0/0)
        if i == 0:
            Tr_0_inv = np.linalg.inv(Tr)

        pose.append((Tr_0_inv.dot(Tr)))
    return np.stack(pose)

## Velodyne in Camera utils

In [0]:
# https://stackoverflow.com/questions/45333780/kitti-velodyne-point-to-pixel-coordinate
def prepare_velo_points(pts3d_raw):
    '''Replaces the reflectance value by 1, and tranposes the array, so
       points can be directly multiplied by the camera projection matrix'''

    pts3d = pts3d_raw
    # Reflectance > 0
    pts3d = pts3d[pts3d[:, 3] > 0 ,:]
    pts3d[:,3] = 1
    return pts3d.transpose()

def project_velo_points_in_img(pts3d, T_cam_velo, Rrect, Prect):
    '''Project 3D points into 2D image. Expects pts3d as a 4xN
       numpy array. Returns the 2D projection of the points that
       are in front of the camera only an the corresponding 3D points.'''

    # 3D points in camera reference frame.
    pts3d_cam = Rrect.dot(T_cam_velo.dot(pts3d))

    # Before projecting, keep only points with z>0 
    # (points that are in fronto of the camera).
    idx = (pts3d_cam[2,:]>=0)
    pts2d_cam = Prect.dot(pts3d_cam[:,idx])
    return pts3d[:, idx], pts2d_cam/pts2d_cam[2,:]

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.vgg16(pretrained=True).to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [0]:
row_list = [["x_displacement", "y_displacement", "sensors_x", "sensors_y", "sensors_d", "sensors_e", "coordinates_src", "depth_src","coordinates_dst", "depth_dst"]]

normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Resize([224, 224]),
   transforms.ToTensor(),
   normalize
])

## Convolutional features utils

In [0]:
def pairwise_distance(X, Y):
    assert len(X.shape) == len(Y.shape)
    N = X.shape[0]
    M = Y.shape[0]
    # D = len(X.shape)
    res = np.zeros([M, N])
    for i in range(M):
        for j in range(N):
            res[i][j] = np.linalg.norm(X[j] - Y[i])
    return res

def match(PD):
    seq = np.arange(PD.shape[0]) # create array of a certain shape from 0 to len
    amin1 = np.argmin(PD, axis=1) # return smallest index from each list
    C = np.array([seq, amin1]).transpose()
    min1 = PD[seq, amin1] # array of the minimum distances
    mask = np.zeros_like(PD) # return an array of zeros of the same shape as DP
    mask[seq, amin1] = 1
    masked = np.ma.masked_array(PD, mask) # PD without the smallest values
    min2 = np.amin(masked, axis=1) # return smallest num in list
    return C, np.array(min2/min1)

def match_max(PD):
    seq = np.arange(PD.shape[0])
    amax1 = np.argmin(PD, axis=1)
    C = np.array([seq, amax1]).transpose()
    return C

## Features with depth utils

In [0]:
def find2perCent(num, listPro):
    listfeatures = []
    Menys = 0.98 * num
    Mes = 1.02 * num 
    for i, el in enumerate(listPro):
        if el < Mes and el > Menys:
            listfeatures.append(i)
    return listfeatures

## Main loop


In [0]:
oxts = []
for pos in dataset.oxts:
    oxt = [i for i in pos[0]]
    oxts.append(oxt)

camera_displacement = []
pose = convertOxtsToPose(oxts)
l = 3 # coordinate axis length
A = np.array([[0, 0, 0, 1], [l, 0, 0, 1], [0, 0, 0, 1], [0, l, 0, 1], [0, 0, 0, 1], [0, 0, l, 1]]).transpose()

In [0]:
index = 0
frames = 5
depth = False
for j, cam3 in enumerate(dataset.cam3):
    B = pose[j].dot(A)
    pts3d = prepare_velo_points(dataset.get_velo(j))
    if j == 0:
        IX = cam3.convert('RGB')
        IX_trand = preprocess(IX)
        IX_trand.unsqueeze_(0)
        sizeX = np.array(IX).shape[1]
        sizeY = np.array(IX).shape[0]
        camScale = np.array(IX.size[:2]) / np.array([224.0, 224.0])
        camScale = camScale[::-1]
        projectionX = project_velo_points_in_img(pts3d, dataset.calib.T_cam3_velo, dataset.calib.R_rect_30,dataset.calib.P_rect_30)

    elif j%frames ==0:
        projectionY = project_velo_points_in_img(pts3d, dataset.calib.T_cam3_velo, dataset.calib.R_rect_30,dataset.calib.P_rect_30)
        IY_trand = preprocess(cam3.convert('RGB'))
        IY_trand.unsqueeze_(0)

        cnn_input = torch.cat((IX_trand, IY_trand), dim=0)
        vis = VisualizeLayers(model,layers='all')
        output = model(cnn_input.to(device))
        # get the intermediate layers output which was passed d  uring initialization
        interm_output = vis.get_interm_output()
        features_pool3 = interm_output['features.16_pooling_MaxPool2d'].cpu()
        
        
        featureMap = []
        DY1 = np.zeros(shape=(784,256))
        pool3_flat = np.reshape(features_pool3[1], [-1, 784]).numpy()
        for i in range(pool3_flat.shape[1]):
            for kernel in pool3_flat:
                featureMap.append(kernel[i])
            DY1[i] = featureMap
            featureMap = []

        DX1 = np.zeros(shape=(784,256))
        pool3_flat = np.reshape(features_pool3[0], [-1, 784]).numpy()
        for i in range(pool3_flat.shape[1]):
            for kernel in pool3_flat:
                featureMap.append(kernel[i])
            DX1[i] = featureMap
            featureMap = []
        
        PD1 = pairwise_distance(DX1, DY1)
        seq = np.array([[i, j] for i in range(28)
                    for j in range(28)], dtype='int32')

        X = np.array(seq, dtype='float32') * 8.0 + 4.0
        Y = np.array(seq, dtype='float32') * 8.0 + 4.0

        # prematch and select points
        C_all, quality = match(PD1)
        theta_max = np.max(quality) # large starting threshold that only confident inliers satisfy
        while np.where(quality >= theta_max)[0].shape[0] <= 128:
            theta_max -= 0.01
        C = C_all[np.where(quality >= theta_max)]
        X, Y = X[C[:, 1]], Y[C[:, 0]]
        # num_matches = C.shape[0]
        Xscaled = X * camScale
        Yscaled = Y * camScale
        if depth: 
            featureWithDepthX = np.zeros(shape=(Xscaled.shape[0], Xscaled.shape[1] +2), dtype=np.float64)
            featureWithDepthY = np.zeros(shape=(Yscaled.shape[0], Yscaled.shape[1] +2), dtype=np.float64)

            projectionX_roundedX = [round(i, 1) for i in projectionX[1][0]]
            projectionX_roundedY = [round(i, 1) for i in projectionX[1][1]]
            projectionY_roundedX = [round(i, 1) for i in projectionY[1][0]]
            projectionY_roundedY = [round(i, 1) for i in projectionY[1][1]]

            for i, feature in enumerate(Xscaled):
                listindX =find2perCent(round(feature[1], 1), projectionX_roundedX)
                listindY =find2perCent(round(feature[0], 1), projectionX_roundedY)
                if len(listindY) > 0 and len(listindX) > 0:
                    cand = set(listindX).intersection(listindY) # detect coordindates 
                    if len(cand)>0:
                        featureWithDepthX[i] = np.append(feature, [True, projectionX[0][0][cand.pop()]])

            for i, feature in enumerate(Yscaled):
                listindX =find2perCent(round(feature[1], 1), projectionY_roundedX)
                listindY =find2perCent(round(feature[0], 1), projectionY_roundedY)
                if len(listindY) > 0 and len(listindX) > 0:
                    cand = set(listindX).intersection(listindY) # detect coordindates 
                    if len(cand)>0:
                        featureWithDepthY[i] = np.append(feature, [True, projectionY[0][0][cand.pop()]])
        else:
            featureWithDepthX = Xscaled
            featureWithDepthY = Yscaled

        difference_listX = []
        difference_listY = []
        difference_listZ = []
        euclideanDistance_list = []

        coordinatesSRC = []
        coordinatesDST = []
        depthListSRC = []
        depthListDST = []

        for i, pnt in enumerate(featureWithDepthX):
            if depth:
                if pnt[2] == True and featureWithDepthY[i][2] == True:
                    src_x = (pnt[1])
                    src_y = (pnt[0])
                    dst_x = (featureWithDepthY[i][1])
                    dst_y = (featureWithDepthY[i][0])

                    difference_listX.append(src_x - dst_x)
                    difference_listY.append(src_y - dst_y)
                    difference_listZ.append(featureWithDepthX[i][3] - featureWithDepthY[i][3])
                    euclideanDistance_list.append(dist.euclidean((src_x, src_y), (dst_x, dst_y)))

                    coordinatesSRC.append(src_x/sizeX)
                    coordinatesSRC.append(src_y/sizeY)
                    depthListSRC.append(featureWithDepthX[i][3])

                    coordinatesDST.append(dst_x/sizeX)
                    coordinatesDST.append(dst_y/sizeY)
                    depthListDST.append(featureWithDepthY[i][3])

            else:
                src_x = (pnt[1])
                src_y = (pnt[0])
                dst_x = (featureWithDepthY[i][1])
                dst_y = (featureWithDepthY[i][0])

                difference_listX.append(src_x - dst_x)
                difference_listY.append(src_y - dst_y)
                euclideanDistance_list.append(dist.euclidean((src_x, src_y), (dst_x, dst_y)))

                depthListSRC.append(1)
                coordinatesSRC.append(src_x/sizeX)
                coordinatesSRC.append(src_y/sizeY)

                depthListDST.append(1)
                coordinatesDST.append(dst_x/sizeX)
                coordinatesDST.append(dst_y/sizeY)

        x_displacement = B[0,0:2][0] - pose[j-frames].dot(A)[0,0:2][0]
        y_displacement = B[1,2:4][0] - pose[j-frames].dot(A)[1,2:4][0]
        index +=1
        
        row_list.append([x_displacement, y_displacement, difference_listX, difference_listY,difference_listZ, euclideanDistance_list, coordinatesSRC, depthListSRC, coordinatesDST, depthListDST])

        IX_trand = IY_trand
        projectionX = projectionY

features_container_Sequential
	 features.0_conv_Conv2d
	 features.1_activation_ReLU
	 features.2_conv_Conv2d
	 features.3_activation_ReLU
	 features.4_pooling_MaxPool2d
	 features.5_conv_Conv2d
	 features.6_activation_ReLU
	 features.7_conv_Conv2d
	 features.8_activation_ReLU
	 features.9_pooling_MaxPool2d
	 features.10_conv_Conv2d
	 features.11_activation_ReLU
	 features.12_conv_Conv2d
	 features.13_activation_ReLU
	 features.14_conv_Conv2d
	 features.15_activation_ReLU
	 features.16_pooling_MaxPool2d
	 features.17_conv_Conv2d
	 features.18_activation_ReLU
	 features.19_conv_Conv2d
	 features.20_activation_ReLU
	 features.21_conv_Conv2d
	 features.22_activation_ReLU
	 features.23_pooling_MaxPool2d
	 features.24_conv_Conv2d
	 features.25_activation_ReLU
	 features.26_conv_Conv2d
	 features.27_activation_ReLU
	 features.28_conv_Conv2d
	 features.29_activation_ReLU
	 features.30_pooling_MaxPool2d
avgpool_pooling_AdaptiveAvgPool2d
classifier_container_Sequential
	 classifier.0_linear_Line

## Create the CSV file

In [0]:
with open('kitti0001_deepFeaturesDepthless-sensors.csv', 'w', newline='') as myfile:
    writer = csv.writer(myfile, quoting=csv.QUOTE_NONNUMERIC, delimiter=',')
    writer.writerows(row_list)
